<a href="https://colab.research.google.com/github/Valentina-Fontanarosa/wellness-forecast-rs/blob/main/Modello_previsione_benessere_giornaliero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy pandas scipy matplotlib seaborn bokeh statsmodels scikit-learn utilsforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.1 MB/s eta 0:00:00


In [2]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Hide warnings
# ==============================================================================
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# Handling and processing of Data
# ==============================================================================
import numpy as np
import pandas as pd

# Handling and processing of Data for Date (time)
# ==============================================================================
import datetime

# Plot
# ==============================================================================
import matplotlib.pyplot as plt

# Modelling with Sklearn
# ==============================================================================
from sklearn.linear_model import LinearRegression

# Define the plot size
# ==============================================================================
from pylab import rcParams
rcParams['figure.figsize'] = (18,7)


# **Caso di studio: previsione del benessere giornaliero di un utente**

L’obiettivo di questo progetto è valutare se, attraverso modelli di deep learning per il forecasting, sia possibile prevedere il livello di benessere giornaliero di un utente, rappresentato da un indice sintetico chiamato ***activity index*** , utilizzando le informazioni relative alla durata del sonno e all’andamento dell’attività fisica, come la velocità del passo.

# **Dataset originale**

Il dataset originale è composto da 4 colonne:

* `user_id`: identificatore dell'utente
* `date`: giorno in cui è stato raccolto il dato aggregato
* `data_type`: intero che identifica la tipologia di dato raccolto
* `data_value`: valore associato al dato raccolto

I diversi valori della variabile `data_type` si riferiscono a dati aggregati raccolti giornalmente dal dispositivo di ogni singolo utente e identificano le tipologie di dato in base al seguente schema:

1.  numero totale di passi effettuati
2.  peso (kg)
3.  BMI (kg/m^2)
4.  pressione sanguigna sistolica (mmHg)
5.  velocità dell'onda sfigmica arteriosa (PWV), (m/s)
6.  PWV healthiness (1: bassa, 2: sano, 3: troppo alta)
7.  frequenza cardiaca media (bpm)
8.  frequenza cardiaca minima (bpm)
9.  frequenza cardiaca massima (bpm)
10. durata del sonno (ore)
11. orario in cui l'utente si è messo a letto
12. orario in cui l'utente si è alzato dal letto
13. numero di volte in cui l'utente si è svegliato durante il sonno
14. durata del tempo in cui l'utente si è svegliato durante il sonno (ore)
15. tempo impiegato dall'utente per addormentarsi (ore)
16. tempo impiegato dall'utente per alzarsi dal letto (ore)
17. durata di sonno leggero (ore)
18. durata di sonno REM (ore)
19. durata di sonno profondo (ore)
20. tipo di attività
21. durata dell'attività (secondi)
22. calorie consumate durante l'attività
23. frequenza cardiaca media durante l'attività (bpm)
24. frequenza cardiaca minima durante l'attività (bpm)
25. frequenza cardiaca massima durante l'attività (bpm)
26. velocità dell'andatura dei passi (passi al minuto)
27. velocità dell'andatura a distanza (km all'ora)

In [3]:
from google.colab import drive
drive.mount('/content/drive')
MYDRIVE = 'drive/MyDrive/Datasets/'

Mounted at /content/drive


In [4]:
df = pd.read_csv(MYDRIVE + "data/data.csv", usecols=[1, 2, 3, 4])
df.columns = ["user_id", "date_local", "data_type", "value"]
print(df.head())

                                    user_id  date_local  data_type     value
0  f3ca4233a8b87472465aa3e0bddc9cc32dd27152  2016-04-01          1  10037.00
1  f3ca4233a8b87472465aa3e0bddc9cc32dd27152  2016-04-01          2     86.51
2  f3ca4233a8b87472465aa3e0bddc9cc32dd27152  2016-04-01          3     28.91
3  f3ca4233a8b87472465aa3e0bddc9cc32dd27152  2016-04-01          7    117.00
4  f3ca4233a8b87472465aa3e0bddc9cc32dd27152  2016-04-01          8    117.00


# **Creazione dataset per la previsione**

E' stato costruito un dataset personalizzato contenente esclusivamente i dati utili all’analisi, con particolare focus sulle variabili legate al sonno e all’attività fisica giornaliera di ciascun utente, in particolare sono state estratte le feature: *sleepduration, bedin, bedout, nbawake, awakeduration, timetosleep, timetowake, remduration, deepduration, activity_type, activity_duration, activity_calories, stepsgaitspeed, distancegaitspeed*.

In [5]:
# Mappatura data_type → nome feature
feature_map = {
    10: "sleepduration", 11: "bedin", 12: "bedout", 13: "nbawake",
    14: "awakeduration", 15: "timetosleep", 16: "timetowake", 18: "remduration",
    19: "deepduration", 20: "activity_type", 21: "activity_duration",
    22: "activity_calories", 26: "stepsgaitspeed", 27: "distancegaitspeed"
}

# Creazione lista di DataFrame per ciascuna feature
dataset_list = [
    df[df['data_type'] == dtype][['user_id', 'date_local', 'value']]
    .rename(columns={'date_local': 'date', 'value': name})
    for dtype, name in feature_map.items()
]

# Merge progressivo su 'user_id' e 'date'
from functools import reduce

dataset = reduce(lambda left, right: pd.merge(left, right, on=['user_id', 'date'], how='outer'), dataset_list)

# Pulizia finale
dataset.dropna(subset=['sleepduration', 'stepsgaitspeed'], inplace=True)
dataset.set_index(['user_id', 'date'], inplace=True)

# Salvataggio
dataset.to_csv(MYDRIVE + 'data/processed_dataset.csv')

In [6]:
dataset = pd.read_csv(MYDRIVE + 'data/processed_dataset.csv', index_col=[0, 1])  # Assicura che user_id e date siano indici
dataset

sleepduration  bedin  \
user_id                                  date                               
0000a344b7eb90edec5801f1562e547b3d3bf1c7 2016-08-08           8.72  22.10   
                                         2016-08-10           8.35  21.90   
                                         2016-08-11           7.15  23.07   
                                         2016-08-12           7.10  22.93   
                                         2016-08-13          10.08  22.90   
...                                                            ...    ...   
fffcad1ee384a6299d314983e5b8bf2fa2d4f41c 2017-03-20           7.87  22.43   
                                         2017-03-21           7.35  23.20   
                                         2017-03-23           5.78  23.58   
                                         2017-03-24           7.25  23.43   
                                         2017-03-28           6.70  23.90   

                                                     bedout  nbawake  \
user_id                                  date                          
0000a344b7eb90edec5801f1562e547b3d3bf1c7 2016-08-08    7.30      3.0   
                                         2016-08-10    7.62      3.0   
                                         2016-08-11    6.83      5.0   
                                         2016-08-12    6.72      4.0   
                                         2016-08-13    9.75      4.0   
...                                                     ...      ...   
fffcad1ee384a6299d314983e5b8bf2fa2d4f41c 2017-03-20    7.03      2.0   
                                         2017-03-21    7.08      2.0   
                                         2017-03-23    5.82      3.0   
                                         2017-03-24    7.17      3.0   
                                         2017-03-28    7.32      3.0   

                                                     awakeduration  \
user_id                                  date                        
0000a344b7eb90edec5801f1562e547b3d3bf1c7 2016-08-08           0.48   
                                         2016-08-10           0.48   
                                         2016-08-11           0.62   
                                         2016-08-12           0.68   
                                         2016-08-13           0.77   
...                                                            ...   
fffcad1ee384a6299d314983e5b8bf2fa2d4f41c 2017-03-20           0.70   
                                         2017-03-21           0.53   
                                         2017-03-23           0.40   
                                         2017-03-24           0.48   
                                         2017-03-28           0.72   

                                                     timetosleep  timetowake  \
user_id                                  date                                  
0000a344b7eb90edec5801f1562e547b3d3bf1c7 2016-08-08         0.10         0.0   
                                         2016-08-10         0.00         0.0   
                                         2016-08-11         0.00         0.0   
                                         2016-08-12         0.05         0.0   
                                         2016-08-13         0.30         0.0   
...                                                          ...         ...   
fffcad1ee384a6299d314983e5b8bf2fa2d4f41c 2017-03-20         0.03         0.0   
                                         2017-03-21         0.00         0.0   
                                         2017-03-23         0.05         0.0   
                                         2017-03-24         0.00         0.0   
                                         2017-03-28         0.00         0.0   

                                                     remduration  \
user_id                                  date                      
0000a344b7eb90edec5801f156

# **Calcolo indice di attività fisica**

La funzione calcola un punteggio (activity_index) per ogni riga del dataset (cioè per ogni giorno di ogni utente), bilanciando:

- Bonus per attività e qualità del sonno, ad esempio chi fa movimento e consuma calorie
- Penalizza poco movimento, poco sonno e risvegli notturni

Questo indice, ottenuto tramite una funzione appositamente definita, combina diverse informazioni relative all’attività fisica e alla qualità del sonno (ad esempio calorie bruciate, durata del sonno e risvegli notturni) in un unico valore numerico utile per la previsione del benessere giornaliero.

In [7]:
def get_activity_score(row):
    score = 0

    # Calorie bruciate
    if not pd.isna(row['activity_calories']):
        if row['activity_calories'] > 500:
            score += 3
        elif row['activity_calories'] > 300:
            score += 2
        elif row['activity_calories'] > 100:
            score += 1

    # Durata attività fisica
    if not pd.isna(row['activity_duration']):
        duration_min = row['activity_duration'] / 60
        if duration_min > 60:
            score += 3
        elif duration_min > 30:
            score += 2
        elif duration_min > 10:
            score += 1

    # Sleepduration
    if not pd.isna(row['sleepduration']):
        if row['sleepduration'] >= 7:
            score += 2
        elif row['sleepduration'] >= 6:
            score += 1

    # Awakeduration
    if not pd.isna(row['awakeduration']):
        if row['awakeduration'] <= 0.5:
            score += 2
        elif row['awakeduration'] <= 1:
            score += 1

    # Deep sleep
    if not pd.isna(row['deepduration']):
        if row['deepduration'] > 1:
            score += 1

    # REM
    if not pd.isna(row['remduration']):
        if 1 <= row['remduration'] <= 2:
            score += 1

    # Passo veloce
    if not pd.isna(row['stepsgaitspeed']):
        if row['stepsgaitspeed'] > 100:
            score += 1

    # Distanza
    if not pd.isna(row['distancegaitspeed']):
        if row['distancegaitspeed'] > 2:
            score += 1

    return score


La colonna activity_index viene calcolata e aggiunta al dataset.

In [8]:
dataset['activity_index'] = dataset.apply(get_activity_score, axis=1)
